# Objective: Normalize Messy Medical Data using Regular Expressions

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

I will be identifing all of the different date variants encoded in this dataset. I will then properly normalize and sort the medical notes by ascending chronological order. 

*This function return a Series of length 500 and dtype int.*
This pandas Series returns the original Series' indices in chronological order

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

The function would return this:

    0    2
    1    4
    2    0
    3    1
    4    3

In [1]:
import pandas as pd

#load the 'dates.txt' file. 
doc = []

#reads each line of the dates.txt file and then adds each new read line as another element in the 'doc' array
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
def date_sorter():
    import pandas as pd
    import numpy as np
    from datetime import datetime
    
    #use various regex's to account for all the different ways a date may be encoded
    #Here is a list the variants I would encounter in this dataset:
        #* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
        #* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009; 
        #* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009 
        #* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009 
        #* Feb 2009; Sep 2009; Oct 2010
        #* 6/2008; 12/2009
        #* 2009; 2010
        
    #regex matches with: 04/20/09; 4/20/09; 4/3/09
    a1_a =df.str.extractall(r'(\d{1,2})[\/-](\d{1,2})[\/-](\d{2})\b')
    #regex matches with: 04/20/2009
    a1_b =df.str.extractall(r'(\d{1,2})[\/-](\d{1,2})[\/-](\d{4})\b')
    a1 = pd.concat([a1_a,a1_b])
    a1.reset_index(inplace=True)
    a1_index = a1['level_0']

    #regex matches with: Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009
    a2 = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.]* )((?:\d{1,2}[?:, -]*)\d{4})')
    a2.reset_index(inplace=True)
    a2_index = a2['level_0']

    #regex matches with:  20 Mar 2009; 20 March 2009;  20 March, 2009; Feb 2009; Sep 2009; Oct 2010
    a3 = df.str.extractall(r'((?:\d{1,2} ))?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[?:, -]* )(\d{4})')
    a3.reset_index(inplace=True)
    a3_index = a3['level_0']

    #regex matches with: 04/20/2009(overlaps with a1); 6/2008; 12/2009
    a4 = df.str.extractall(r'(\d{1,2})[\/](\d{4})')
    a4.reset_index(inplace=True)
    a4_index = a4['level_0']

    #strings matched by the a4 pattern are also captured by the a1 pattern.
    #I want to only isolate the a4 matches that do not overlap with the a1 matches
    save=[]
    for i in a4_index:
        if not(i in a1_index.values):
             save.append(i)
    save = np.asarray(save)
    a4 = a4[a4['level_0'].isin(save)]
    
    #regex matches with: 
        #* 04/20/2009;
        #* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
        #* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
        #* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
        #* Feb 2009; Sep 2009; Oct 2010
        #* 6/2008; 12/2009
        #* 2009; 2010 
    a5_a= df.str.extractall(r'[a-z]?[^0-9](\d{4})[^0-9]')
    
    #regex matches with:
        #* 04/20/2009;
        #* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009
        #* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
        #* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
        #* Feb 2009; Sep 2009; Oct 2010
        #* 6/2008; 12/2009
        #* 2009; 2010 
    a5_b = df.str.extractall(r'^(\d{4})[^0-9]')
    a5 = pd.concat([a5_a,a5_b])
    a5.reset_index(inplace=True)
    a5_index = a5['level_0']

    #regex matches in a5 overlaps with regex matches in a2, a3, and a4.
    #I only want the a5 index matchs that do not overlap with a2,a3, or a4
    save=[]
    for i in a5_index:
        if not((i in a2_index.values) | (i in a3_index.values) | (i in a4_index.values)):
             save.append(i)
    save = np.asarray(save)
    a5 = a5[a5['level_0'].isin(save)]

    #each line is now matched with the correct date variant
    #I create additional columns containing the month, day, and year isolated
    a1.columns = ['level_0', 'match', 'month', 'day', 'year']
    a1['year'] = a1['year'].apply(str)
    a1['year'] = a1['year'].apply(lambda x: '19'+x if len(x)<=2 else x)

    a2[1] = a2[1].apply(lambda x:x.replace(',',''))
    a2['day'] = a2[1].apply(lambda x:x.split(' ')[0])
    a2['year'] = a2[1].apply(lambda x:x.split(' ')[1])
    a2 = a2.drop(1, axis = 1)
    a2.columns = ['level_0', 'match', 'month', 'day', 'year']
    a2['month'] = a2.month.apply(lambda x: x[:3])
    a2['month'] = pd.to_datetime(a2.month, format='%b').dt.month

    a3.columns = ['level_0', 'match', 'day', 'month', 'year']
    a3['day']=a3['day'].replace(np.nan,1)
    a3['month'] = a3.month.apply(lambda x: x[:3])
    a3['month'] = pd.to_datetime(a3.month, format='%b').dt.month

    a4.columns = ['level_0','match','month','year']
    a4['day'] = 1

    a5.columns=['level_0','match','year']
    a5['day']=1
    a5['month']=1
    
    #combine all the normalized strings together
    results = pd.concat([a1,a2,a3,a4,a5])
    results['date'] =pd.to_datetime(results['month'].apply(str)+'/'+results['day'].apply(str)+'/'+results['year'].apply(str))
    results = results.sort_values(by = 'level_0')
    results = results.set_index('level_0')
    final = results['date']

    n = np.arange(500)
    sorted_dates = sorted(enumerate(final),key=lambda x:x[1])

    return pd.Series(list(i[0] for i in sorted_dates),n)

date_sorter().head()

0     9
1    84
2     2
3    53
4    28
dtype: int64